In [3]:
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model

from keras.applications.efficientnet_v2 import EfficientNetV2B1
from keras.applications.vgg19 import preprocess_input
from keras.applications.densenet import DenseNet121
from keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow import keras
from tensorflow.keras import layers, models, callbacks
import tensorflow as tf
from keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

In [ ]:
import os
import kaggle


# Set up Kaggle API credentials
download_path = '/tmp'
# Downloading the data
kaggle.api.dataset_download_files('paultimothymooney/chest-xray-pneumonia', path=download_path, unzip=True)

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import shutil
import random

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/tmp/chest_xray/'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

dataset_path= '/tmp/chest_xray/'
new_dataset_path = '/tmp/chest_xray_new/'
if not os.path.exists(new_dataset_path):
   for split in ['train', 'val', 'test']:
       for cls in ['NORMAL', 'PNEUMONIA']:
           os.makedirs(f'{new_dataset_path}/{split}/{cls}', exist_ok=True)

   for cls in ['NORMAL', 'PNEUMONIA']:
       all_files = []
       for split in ['train', 'val', 'test']:
           source_folder = f'{dataset_path}/{split}/{cls}'
           files = os.listdir(source_folder)
           all_files.extend([(file, source_folder) for file in files])

       random.shuffle(all_files)

       train_files = all_files[:int(len(all_files)*0.8)]
       val_files = all_files[int(len(all_files)*0.8):int(len(all_files)*0.9)]
       test_files = all_files[int(len(all_files)*0.9):]

       for file, source_folder in train_files:
           dest = f'{new_dataset_path}/train/{cls}/{file}'
           shutil.copy(f'{source_folder}/{file}', dest)

       for file, source_folder in val_files:
           dest = f'{new_dataset_path}/val/{cls}/{file}'
           shutil.copy(f'{source_folder}/{file}', dest)

       for file, source_folder in test_files:
           dest = f'{new_dataset_path}/test/{cls}/{file}'
           shutil.copy(f'{source_folder}/{file}', dest) 

In [4]:
input_shape = (224, 224, 3)

train_path =  '/tmp/chest_xray_new/train'
test_path =  '/tmp/chest_xray_new/test'
valid_path =  '/tmp/chest_xray_new/val'


In [19]:
input_tensor = Input(shape=input_shape)
model = DenseNet121(include_top= False , # remove  the 3 fully-connected layers at the top of the network
                input_tensor=input_tensor, 
                input_shape= input_shape,
                pooling ='avg')
model.load_weights('brucechou1983_CheXNet_Keras_0.3.0_weights.h5', skip_mismatch=True, by_name=True)


In [9]:
input_tensor = Input(shape=input_shape)
Efv21 = EfficientNetV2B1(include_top=False, input_tensor=input_tensor, weights="imagenet")


In [5]:

# don't train existing weights
for layer in Efv21.layers:
    layer.trainable = False

In [6]:
# our layers - 
x = Flatten()(Efv21.output)

In [7]:
prediction = Dense(2, activation='softmax')(x)

In [8]:
model = Model(inputs=Efv21.input, outputs=prediction)

In [20]:
model.summary()

Model: "densenet121"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_9 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 zero_padding2d_10 (ZeroPadding  (None, 230, 230, 3)  0          ['input_9[0][0]']                
 2D)                                                                                              
                                                                                                  
 conv1/conv (Conv2D)            (None, 112, 112, 64  9408        ['zero_padding2d_10[0][0]']      
                                )                                                       

In [27]:
from tensorflow.keras import callbacks, optimizers


checkpoint_callback = callbacks.ModelCheckpoint(filepath="tmp/efv21_best_model.keras", monitor='val_accuracy', save_best_only=True)

from tensorflow.keras.callbacks import EarlyStopping

# Define early stopping callback
early_stopping_callback = EarlyStopping(monitor='val_accuracy', patience=10,mode='max', verbose=1 )

model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [29]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
valid_datagen = ImageDataGenerator(rescale = 1./255)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [30]:
training_set = train_datagen.flow_from_directory(train_path,
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 4684 images belonging to 2 classes.


In [31]:
valid_set = valid_datagen.flow_from_directory(test_path,
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 587 images belonging to 2 classes.


In [32]:
test_set = test_datagen.flow_from_directory(valid_path,
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 585 images belonging to 2 classes.


In [33]:
import math
import numpy as np
batch_size=32

number_of_examples = len(test_set.filenames)
number_of_generator_calls = math.ceil(number_of_examples / batch_size)

test_labels = []

# Iterate through the generator calls
for i in range(0, int(number_of_generator_calls)):
    # Get the labels for the current batch
    batch_labels = np.array(test_set[i][1])
    test_labels.extend(batch_labels)
    


In [ ]:
# fit the model

r = model.fit(
  training_set,
  validation_data=valid_set,
  epochs=3,
 callbacks=[checkpoint_callback,early_stopping_callback]
)


Epoch 1/3


In [ ]:
# plot the loss
plt.plot(r.history['loss'], label='train loss')
plt.plot(r.history['val_loss'], label='val loss')
plt.legend()
plt.show()
plt.savefig('LossVal_loss')

# plot the accuracy
plt.plot(r.history['accuracy'], label='train acc')
plt.plot(r.history['val_accuracy'], label='val acc')

plt.legend()
plt.show()
plt.savefig('AccVal_acc')

In [35]:
test_loss, test_acc = model.evaluate(test_set)
print('Test accuracy:', test_acc)

InvalidArgumentError: Graph execution error:

Detected at node 'categorical_crossentropy/mul' defined at (most recent call last):
    File "C:\Users\301212298\.pyenv\pyenv-win\versions\3.10.0\lib\runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "C:\Users\301212298\.pyenv\pyenv-win\versions\3.10.0\lib\runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "C:\Users\301212298\AppData\Local\pypoetry\Cache\virtualenvs\neural-network-winter-2024-final-project-c-p6kOt2mb-py3.10\lib\site-packages\ipykernel_launcher.py", line 18, in <module>
      app.launch_new_instance()
    File "C:\Users\301212298\AppData\Local\pypoetry\Cache\virtualenvs\neural-network-winter-2024-final-project-c-p6kOt2mb-py3.10\lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
      app.start()
    File "C:\Users\301212298\AppData\Local\pypoetry\Cache\virtualenvs\neural-network-winter-2024-final-project-c-p6kOt2mb-py3.10\

In [28]:
from sklearn.metrics import classification_report


y_pred = model.predict(test_set)

print(classification_report(np.argmax(test_labels,axis=1), np.argmax(y_pred,axis=1)))

19/19 [==============================] - 31s 1s/step
              precision    recall  f1-score   support

           0       0.00      0.00      0.00     158.0
           1       0.00      0.00      0.00     427.0
         143       0.00      0.00      0.00       0.0
         162       0.00      0.00      0.00       0.0
         170       0.00      0.00      0.00       0.0
         209       0.00      0.00      0.00       0.0
         295       0.00      0.00      0.00       0.0
         303       0.00      0.00      0.00       0.0
         409       0.00      0.00      0.00       0.0
         437       0.00      0.00      0.00       0.0
         473       0.00      0.00      0.00       0.0
         507       0.00      0.00      0.00       0.0
         535       0.00      0.00      0.00       0.0
         545       0.00      0.00      0.00       0.0
         546       0.00      0.00      0.00       0.0
         552       0.00      0.00      0.00       0.0
         560       0.00     

C:\Users\301212298\AppData\Local\pypoetry\Cache\virtualenvs\neural-network-winter-2024-final-project-c-p6kOt2mb-py3.10\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\301212298\AppData\Local\pypoetry\Cache\virtualenvs\neural-network-winter-2024-final-project-c-p6kOt2mb-py3.10\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\301212298\AppData\Local\pypoetry\Cache\virtualenvs\neural-network-winter-2024-final-project-c-p6kOt2mb-py3.10\lib\site-packages\sklearn\metrics\_classification.py:1509: Un

In [ ]:
import tensorflow as tf

from keras.models import load_model

model.save('model_efv21.h5')

In [ ]:
def show_image(img_name):
  img = plt.imread(img_name)
  plt.imshow(img)
  plt.show()